<a href="https://colab.research.google.com/github/ajaykumarmehra/Toxic-Comment-Detection-and-Classification/blob/main/LSTM_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajaykumarmehra","key":"9ff7ad499cf023ccffa8bdff18cd9c82"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              215KB  2021-03-12 08:10:45            308  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            358  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            275  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            133  
fatiimaezzahra/f

In [ ]:
! kaggle datasets download -d thanakomsn/glove6b300dtxt

100% 385M/386M [00:06<00:00, 65.7MB/s]
100% 386M/386M [00:06<00:00, 66.0MB/s]


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
! unzip glove6b300dtxt.zip -d "/content/gdrive/My Drive/dataset_toxic_comment/embeddings"

unzip:  cannot find or open glove6b300dtxt.zip, glove6b300dtxt.zip.zip or glove6b300dtxt.zip.ZIP.


In [4]:
import pandas as pd
import numpy as np

In [5]:
path = '/content/gdrive/My Drive/dataset_toxic_comment/'
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
labels = pd.read_csv(path+"test_labels.csv")

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train.isnull().any(),test.isnull().any()

(id               False
 comment_text     False
 toxic            False
 severe_toxic     False
 obscene          False
 threat           False
 insult           False
 identity_hate    False
 dtype: bool, id              False
 comment_text    False
 dtype: bool)

In [12]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [10]:
from keras.preprocessing.text import Tokenizer

In [15]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [16]:
from keras.preprocessing.sequence import pad_sequences

In [17]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [18]:
gl_path = '/content/gdrive/My Drive/dataset_toxic_comment/embeddings/glove.6B.300d.txt'

In [19]:
import gensim.models.keyedvectors as word2vec
import gc

In [20]:
def loadEmbeddingMatrix():
  EMBEDDING_FILE=gl_path
  embed_size = 300
  
  embeddings_index = dict()
  f = open(EMBEDDING_FILE)
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
  f.close()
  print('Found %s word vectors.' % len(embeddings_index))
      
  gc.collect()
  all_embs = np.stack(list(embeddings_index.values()))
  emb_mean,emb_std = all_embs.mean(), all_embs.std()
  
  nb_words = len(tokenizer.word_index)
  embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
  gc.collect()

  embeddedCount = 0
  for word, i in tokenizer.word_index.items():
      i-=1
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None: 
          embedding_matrix[i] = embedding_vector
          embeddedCount+=1
  print('total embedded:',embeddedCount,'common words')
  
  del(embeddings_index)
  gc.collect()
  
  return embedding_matrix

In [21]:
embedding_matrix = loadEmbeddingMatrix()

Found 400000 word vectors.
total embedded: 95685 common words


In [22]:
embedding_matrix.shape

(210337, 300)

In [23]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model

In [24]:
inp = Input(shape=(maxlen, ))

In [25]:
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)

In [26]:
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

In [27]:
import keras.metrics as metrics

In [28]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[
                          metrics.MeanSquaredError(),
                          metrics.AUC(),
                  ])

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 300)          63101100  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 120)          173280    
_________________________________________________________________
global_max_pooling1d (Global (None, 120)               0         
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0     

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_t,y,test_size=0.2,random_state=1) 

In [31]:
batch_size = 64
epochs = 4
hist = model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/4
1995/1995 [==============================] - 2152s 1s/step - loss: 0.1318 - mean_squared_error: 0.0345 - auc: 0.8353
Epoch 2/4
1995/1995 [==============================] - 2154s 1s/step - loss: 0.0607 - mean_squared_error: 0.0159 - auc: 0.9663
Epoch 3/4
1995/1995 [==============================] - 2165s 1s/step - loss: 0.0544 - mean_squared_error: 0.0146 - auc: 0.9748
Epoch 4/4
1995/1995 [==============================] - 2159s 1s/step - loss: 0.0493 - mean_squared_error: 0.0135 - auc: 0.9803


In [32]:
preds_train = model.predict(X_train)

In [33]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

print(roc_auc_score(y_train, preds_train))

0.9856213137987626


In [34]:
preds_val = model.predict(X_val)
print(roc_auc_score(y_val, preds_val))

0.9751773190901609


In [35]:
labels = labels[classes]
sum_labels=np.sum(labels.values,axis=1)
# print(sum_labels)
idx=sum_labels>=0
y_test = labels[idx]
X_test = X_te[idx]

In [37]:
preds_test = model.predict(X_test)
print(roc_auc_score(y_test, preds_test))

0.9666645630149224


In [38]:
# Save the entire model as a SavedModel.
model.save('/content/gdrive/My Drive/dataset_toxic_comment/saved_model/LSTM_Glove_4Epoch') 

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/dataset_toxic_comment/saved_model/LSTM_Glove_4Epoch/assets


In [39]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


In [40]:
import keras
new_model = keras.models.load_model('/content/gdrive/My Drive/dataset_toxic_comment/saved_model/LSTM_Glove_4Epoch')

# Check its architecture
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 300)          63101100  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 120)          173280    
_________________________________________________________________
global_max_pooling1d (Global (None, 120)               0         
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0     

In [41]:
labels = labels[classes]
sum_labels=np.sum(labels.values,axis=1)
# print(sum_labels)
idx=sum_labels>=0
y_test = labels[idx]
X_test = X_te[idx]

In [42]:
preds_test = new_model.predict(X_test)

In [43]:
from sklearn.metrics import roc_auc_score, hamming_loss, accuracy_score, log_loss
import numpy as np
print(roc_auc_score(y_test, preds_test))
print("ROC AUC Score: ", roc_auc_score(y_test, preds_test))
print("Hamming Loss: ", hamming_loss(y_test, (preds_test > 0.5)))
print("Log Loss: ", log_loss(y_test, preds_test))
print("Accuracy Score: ", accuracy_score(y_test, (preds_test > 0.5)))

0.9666645630149224
ROC AUC Score:  0.9666645630149224
Hamming Loss:  0.02574582929548699
Log Loss:  0.29125949122474454
Accuracy Score:  0.8979336646972397


In [ ]:
# from sklearn.metrics import multilabel_confusion_matrix
# preds_test = (preds_test > 0.5) 
# df = pd.DataFrame(preds_test, columns = ['toxic','severe_toxic','obscence', 'threat', 'insult', 'identity_hate'])
# matrix = multilabel_confusion_matrix(y_test, df)
# print(matrix)

Above code is used to save and load model again 